# GDELT 1.0 Code (skip)

Will have to integrate this into GDELT 2.0.  Headers are different.  GDELT 1.0 goes back to 1979.  2.0 only goes back to Feb 2015

![](http://data.gdeltproject.org/dailymaps_noaasos/spinningglobe.gif)

In [ ]:
from IPython.display import Image
Image(url='../utils/images/spinningglobe.gif')

In [79]:
import requests
import lxml.html as lh

gdelt_base_url = 'http://data.gdeltproject.org/events/'
gdelt_gkg_url = 'http://api.gdeltproject.org/api/v1/gkg_geojson'
# get the list of all the links on the gdelt file page
page = requests.get(gdelt_base_url+'index.html')
doc = lh.fromstring(page.content)
link_list = doc.xpath("//*/ul/li/a/@href")

# separate out those links that begin with four digits 
file_list = [x for x in link_list if str.isdigit(x[0:4])]


In [2]:
masterListUrl = 'http://data.gdeltproject.org/gdeltv2/masterfilelist.txt'
directory = requests.get(masterListUrl)
results = directory.content.split('\n')

In [233]:
results;

In [234]:
import pandas as pd

pd.options.display.max_rows = 200
# df = pd.DataFrame(data.json())

In [ ]:
df['coords'] = df.features.apply(lambda row: row['geometry']['coordinates'])
df['lat'] = df.features.apply(lambda row: row['geometry']['coordinates'][1])
df['lon'] = df.features.apply(lambda row: row['geometry']['coordinates'][0])
df['name'] = df.features.apply(lambda row: row['properties']['name'])
df['pubdate'] = df.features.apply(lambda row: row['properties']['urlpubtimedate'])
df['urltone'] = df.features.apply(lambda row: row['properties']['urltone'])
df['mentionedNames'] = df.features.apply(lambda row: row['properties']['mentionednames'])
df['mentioinedThemes'] = df.features.apply(lambda row: row['properties']['mentionedthemes'])
df['url'] = df.features.apply(lambda row: row['properties']['url'])

# GDELT 2.0 Access

In [1]:
import requests
import pandas as pd
import numpy as np
import re
from dateutil.parser import parse

# Logic for GDELT module

Enter a date or date range.  GDELT 2.0 only goes to Feb 18 2015.  GDELT 1.0 goes back to 1979.  

Convert the entered date or date range to string, search for string in the master df list.  Use the tblType parameter to pull the correct table(s).  

* default is take current time and most recent file
* enter historical date; defaults to last record of day
    * parse
    * add feature to enter time for historical and pull closest 15 minute file
    * date range will pull last file for each day and concatenate into single dataframe
    
choose a database
*  Select between events, event mentions or gkg

return it as a python or R dataframe
*  use the feather library for Python

*********************


# URLS

The main urls that we need to hit to return data.

In [85]:
masterListUrl = 'http://data.gdeltproject.org/gdeltv2/masterfilelist.txt'
baseUrl = 'http://data.gdeltproject.org/gdeltv2/'

# Parameters and Global Variables

Section contains variables that will be `self.` objects in the classes.

In [86]:

'''
Listing of all GDELT 15 minute dumps. Code retrieves the list,
splits it on the new line character, and then splits on the space. 
We delete the last entry because it's empty.  
'''
directory = requests.get(masterListUrl)
clean = directory.content.split('\n')
clean = map(lambda x: x.split(' '),clean)
del clean[-1]

"""
Setting up the master list as dataframe for querying
this will be inside the class
"""
masterdf = pd.DataFrame(clean)
masterdf.fillna('',inplace=True)

In [87]:
# table type = tblType
graph = 'gkg'
events = 'events' # includes new GDELT 2.0 mentions table; merged on globaleventid

tblType = events  # default to events db


## Date Parameters that will be entered

Location to hold testing spot for all the different type of parameters that can be entered.

In [153]:
defaultDateEntry = "" # string
stringDateEntry = " 2016 09 18" # string
historicalDateEntry = "2015 02 25" #string
errorDate = "What in the heck" # error string
listOfdates = ['Sep 1 2016','2016 09 24'] # list, len 2
moreThanTwo= ['Sept 20 2016','June 3 2011','January 1, 2013'] # list, len greater than 2d

date = defaultDateEntry
time = ""

In [7]:
date

['Sep 1 2016', '2016 09 24']

## Setting the values for the headers

Headers are set based on `tblType` value passed in.  Will default to the events DB headers.  

In [88]:
gkgHeaders = pd.read_csv(
    '../utils/schema_csvs/GDELT_2.0_gdeltKnowledgeGraph_Column_Labels_Header_Row_Sep2016.tsv',
    delimiter='\t',usecols=['tableId','dataType','Description']
    )
gkgHeaders.tableId.tolist();

eventsDbHeaders = pd.read_csv('../utils/schema_csvs/GDELT_2.0_Events_Column_Labels_Header_Row_Sep2016.csv',
                         delimiter=',',usecols=['tableId','dataType','Description'])
eventsDbHeaders.tableId.tolist();

mentionsHeaders = pd.read_csv('../utils/schema_csvs/GDELT_2.0_eventMentions_Column_Labels_Header_Row_Sep2016.tsv',
                         delimiter='\t',usecols=['tableId','dataType','Description'])
mentionsHeaders.tableId.tolist();



**************************


# Checking Inputs of functions and parameters

We need to see how many dates are passed into the function.  Use the logic above. 

In [72]:
import traceback,sys
import datetime
from dateutil.parser import parse
import numpy as np

def dateInputCheck(date):
    if isinstance(date,str):
        if date != "":
            if parse(date) > datetime.datetime.now():
                raise ValueError('Your date is greater than the current date.  Please enter a relevant date.')
            elif parse(date)<parse('Feb 18 2015'):
                raise ValueError('GDELT 2.0 only supports \'Feb 18 2015 - Present\' queries currently. Try another date.')

    elif isinstance(date,list):
        if len(date)==1:
            try:
                if parse("".join(date)) > datetime.datetime.now():
                    raise ValueError('Your date is greater than the current date.  Please enter a relevant date.')
                elif parse("".join(date)) < parse('Feb 18 2015'):
                    raise ValueError('GDELT 2.0 only supports \'Feb 18 2015 - Present\' queries currently. Try another date.')
            except:
                exc_type, exc_value, exc_traceback = sys.exc_info()
                traceback.print_tb(exc_traceback, limit=1, file=sys.stdout)
                traceback.print_exception(exc_type, exc_value, exc_traceback,
                                          limit=2, file=sys.stdout)
                raise ValueError("One or more of your input date strings does not parse to a date format. Check input.")

        
        elif len(date)==2:
            try:
                map(parse,date)
            except Exception as exc:
                exc_type, exc_value, exc_traceback = sys.exc_info()
                traceback.print_tb(exc_traceback, limit=1, file=sys.stdout)
                traceback.print_exception(exc_type, exc_value, exc_traceback,
                                          limit=2, file=sys.stdout)
                raise ValueError("One or more of your input date strings does not parse to a date format. Check input.")

            if bool(parse(date[0])<parse(date[1])) == False:
                raise ValueError('Start date greater than end date. Check date strings.')
                
            if np.all(np.logical_not(np.array(map(parse,date))> datetime.datetime.now())) == False:
                raise ValueError("One of your dates is greater than the current date. Check input date strings.")

            
        elif len(date)>2:

            try:
                map(parse,date)
            except Exception as exc:
                exc_type, exc_value, exc_traceback = sys.exc_info()
                traceback.print_tb(exc_traceback, limit=1, file=sys.stdout)
                traceback.print_exception(exc_type, exc_value, exc_traceback,
                                          limit=2, file=sys.stdout)
                raise ValueError("One or more of your input date strings does not parse to a date format. Check input.")
                
            if np.all(np.logical_not(np.array(map(parse,date))> datetime.datetime.now())) == False:
                raise ValueError("One or more of your input date strings does not parse to a date format. Check input.")

        

In [74]:
date=['2016 9 12']
dateInputCheck(date)

### Checking the tblType input

In [171]:

# gets the urls from array
# resultMaster = vectorizedUrlFinder(UrlFinder,datesToPull)


def tblCheck(tbl):
    '''Checking the input of tblType.'''
    if tbl == 'events' or tbl == '' or tbl == 'mentions':
        resultsUrlList = resultMaster[2][resultMaster[2].str.contains('export|mentions')]
    elif tbl == 'gkg':
        resultsUrlList = resultMaster[2][resultMaster[2].str.contains('gkg')]
    else:
        raise ValueError ("Incorrect parameter \'{0}\' entered.  Did you mean to use \'{0}\' as the parameter?\nPlease check your \'tblType\' parameters.".format(tblType))
    return resultsUrlList

*************

# Date Functionality (Date ranges)

Use the numpy date range functionality to create strings of dates between ranges in a list.  Then, use the dateutil tool to parse those strings into the correct format.  Then run a query for each date, return the dataframe, and concatenate into a single one.

* Logic
    * If length of passed in date less than zero, raise error
    * If length is equal to one, find that one date's table or graph
    * If length equal to two:
        * if dates are chronological, covert to numpy range and pull all tables or graphs, but raise warning for long ranges
        * if dates are not chronological, get individual dates
    * If length greater than two, get the individual dates
        * initially, return the latest time
        * add option to return closest 15 minute interval to passed in time

## Code Pieces and Functions

In [6]:
# numpy example of ranging the date
np.arange('2016-08-01', '2016-09-16', dtype='datetime64[D]');

In [3]:
#############################################
# Parse the date
#############################################


from dateutil.parser import parse
import pandas as pd
import numpy as np 
import requests
import datetime



def parse_date(var):
    """Return datetime object from string."""
    
    try:
        return np.where(isinstance(parse(var),datetime.datetime),
                 parse(var),"Error")             
    except:
        return "You entered an incorrect date.  Check your date format."


# def gdelt_timeString(dateInputVar):
#     """Convert date to GDELT string file format for query."""
    
#     multiplier = dateInputVar.tolist().minute / 15
#     multiple = 15 * multiplier
#     queryDate = np.where(
#             multiplier > 1,dateInputVar.tolist().replace(
#             minute=0, second=0) + datetime.timedelta(
#             minutes=multiple),
#             dateInputVar.tolist().replace(
#             minute=0, second=0,microsecond=0000)
#             )
    
#     # Check for date equality on historical query
#     modifierTip = datetime.datetime.now().replace(
#         hour=0,minute=0,second=0,microsecond=0
#         ) == queryDate.tolist().replace(
#         hour=0,minute=0,second=0,microsecond=0
#         )
    
#     # Based on modifier, get oldest file for historical query
#     queryDate = np.where(
#         modifierTip==False,
#         queryDate.tolist().replace(
#             hour=23,
#             minute=45,
#             second=00,
#             microsecond=0000
#             ),queryDate
#         )
    
# #     print modifierTip
#     return queryDate.tolist().strftime("%Y%m%d%H%M%S")

#############################################
# Match parsed date to GDELT master list
#############################################

# def match_date(dateString):
#     """Return dataframe with GDELT data for matching date"""
    
#     masterListUrl = 'http://data.gdeltproject.org/gdeltv2/masterfilelist.txt'
#     directory = requests.get(masterListUrl)
#     results = directory.content.split('\n')
#     results = map(lambda x: x.split(' '),results)
#     masterListdf = pd.DataFrame(results)
#     return masterListdf[
#         masterListdf[2].str.contains(
#             dateString
#             )==True
#         ]
    
def dateformatter(datearray):
    """Function to format strings for numpy arange"""
    return parse(datearray).strftime("%Y-%m-%d")
    
def dateRanger(originalArray):
    """Function to vectorize date formatting function.
    Creates datetime.date objects for each day in the range
    and stores in a numpy array.
    
    Example
    
    Parameters
        ----------
        X : {array-like, sparse matrix}, shape (n_samples, n_features)
            Input data, where ``n_samples`` is the number of samples and
            ``n_features`` is the number of features.
    Returns
    -------
    self : object
        Returns self.
    """
    if isinstance(originalArray,str):
        """Check user input to retrieve date query."""
    
        return np.where(len(originalArray)==0,np.array(datetime.datetime.now()),
                 parse_date(originalArray))
    
    elif isinstance(originalArray,list):
        if len(originalArray)==1:
            return np.array(parse("".join(originalArray)))
        elif len(originalArray)>2:
            return np.array(map(parse,originalArray),dtype='datetime64[D]')
        else:
            cleaner = np.vectorize(dateformatter)
            converted = cleaner(originalArray).tolist()
            dates = np.arange(converted[0],converted[1],dtype='datetime64[D]')
            dates = np.append(dates,np.datetime64(datetime.date.today())) # numpy range is not endpoint inclusive
            return dates

def gdeltRangeString(element):
    if element == datetime.date.today():
        multiplier = datetime.datetime.now().minute / 15
        multiple = 15 * multiplier
        converted = datetime.datetime.now().replace(minute=multiple,second=0)
    else:
        converted = (datetime.datetime.combine(element,datetime.time.min) + 
            datetime.timedelta(
                                minutes=45,hours=23
                                )
                               )
    return converted.strftime('%Y%m%d%H%M%S')



def vectorizer(function,dateArray):
    helper = np.vectorize(function)
    return helper(dateArray.tolist()).tolist()

# Finds the urls from an array of dates

def UrlFinder(targetDate):
    return masterdf[masterdf[2].str.contains(targetDate)]

def vectorizedUrlFinder(function,urlList):
    helper=np.vectorize(function)
    return pd.concat(helper(urlList).tolist())

def downloadVectorizer(function,urlList):
    '''
    test2 = downloadVectorizer(downloadAndExtract,b)
    test2.columns=gkgHeaders.tableId.tolist()
    '''
    helper=np.vectorize(function)
    return pd.concat(helper(urlList).tolist())


### Working Examples for Single Date Functionality

In [4]:
date = '2016 9 12'

vectorizer(gdeltRangeString,dateRanger(date))

'20160912234500'

### Working Examples of Date Range Functionality

In [396]:
date=['2016 09 01','2016 09 24']
(dateRanger(date))

array(['2016-09-01', '2016-09-02', '2016-09-03', '2016-09-04',
       '2016-09-05', '2016-09-06', '2016-09-07', '2016-09-08',
       '2016-09-09', '2016-09-10', '2016-09-11', '2016-09-12',
       '2016-09-13', '2016-09-14', '2016-09-15', '2016-09-16',
       '2016-09-17', '2016-09-18', '2016-09-19', '2016-09-20',
       '2016-09-21', '2016-09-22', '2016-09-23', '2016-09-25'], dtype='datetime64[D]')

In [83]:
# converts to gd
datesToPull = vectorizer(gdeltRangeString,dateRanger(date))

In [84]:
# gets the urls from array
resultMaster = vectorizedUrlFinder(UrlFinder,datesToPull)

NameError: global name 'masterdf' is not defined

In [81]:
resultMaster

NameError: name 'resultMaster' is not defined

## Testing Area for Dates; Above is good, below is experimental

In [150]:
tblCheck('gkg')

160895    http://data.gdeltproject.org/gdeltv2/201609012...
161183    http://data.gdeltproject.org/gdeltv2/201609022...
161471    http://data.gdeltproject.org/gdeltv2/201609032...
161756    http://data.gdeltproject.org/gdeltv2/201609042...
162044    http://data.gdeltproject.org/gdeltv2/201609052...
162332    http://data.gdeltproject.org/gdeltv2/201609062...
162620    http://data.gdeltproject.org/gdeltv2/201609072...
162908    http://data.gdeltproject.org/gdeltv2/201609082...
163196    http://data.gdeltproject.org/gdeltv2/201609092...
163484    http://data.gdeltproject.org/gdeltv2/201609102...
163772    http://data.gdeltproject.org/gdeltv2/201609112...
164060    http://data.gdeltproject.org/gdeltv2/201609122...
164348    http://data.gdeltproject.org/gdeltv2/201609132...
164639    http://data.gdeltproject.org/gdeltv2/201609142...
164924    http://data.gdeltproject.org/gdeltv2/201609152...
165212    http://data.gdeltproject.org/gdeltv2/201609162...
165500    http://data.gdeltproject.org/g

In [73]:
for l in masterdf[2][masterdf[2].str.contains(datesToPull[20])]:
    print l

http://data.gdeltproject.org/gdeltv2/20160921234500.export.CSV.zip
http://data.gdeltproject.org/gdeltv2/20160921234500.mentions.CSV.zip
http://data.gdeltproject.org/gdeltv2/20160921234500.gkg.csv.zip


In [45]:
test2.reset_index(drop=True,inplace=True)

In [46]:
test2

,GKGRECORDID,DATE,SourceCollectionIdentifier,SourceCommonName,DocumentIdentifier,Counts,V2Counts,Themes,V2Themes,Locations,...,GCAM,SharingImage,RelatedImages,SocialImageEmbeds,SocialVideoEmbeds,Quotations,AllNames,Amounts,TranslationInfo,Extras
0,20160901234500-0,20160901234500,2,BBC Monitoring,Facebook in Russian and Uzbek /BBC Monitoring/...,NaN,NaN,MEDIA_SOCIAL;GENERAL_HEALTH;MEDICAL;TAX_ETHNIC...,"GENERAL_HEALTH,30;MEDICAL,30;MEDIA_SOCIAL,10;M...",1#Uzbekistan#UZ#UZ#41#64#UZ,...,"wc:289,c12.1:32,c12.10:13,c12.12:7,c12.13:1,c1...",NaN,NaN,NaN,NaN,"487|265||Dear friends , I sincerely apologise ...","Uzbek President Islam Karimov,130;Islam Karimo...",NaN,NaN,NaN
1,20160901234500-1,20160901234500,1,nigeriasun.com,http://www.nigeriasun.com/index.php/sid/247252977,NaN,NaN,NaN,NaN,"4#Mumbai, Maharashtra, India#IN#IN16#18.975#72...",...,"wc:1024,c1.1:2,c1.3:1,c12.1:52,c12.10:103,c12....",http://www.nigeriasun.comhttp://cdn.bignewsnet...,http://cdn.bignewsnetwork.com/ani1472717432.jpg,NaN,NaN,NaN,"Hewlett Packard Enterprise,232;Country Directo...","8,introduces a unified architecture,267;8,prov...",NaN,NaN
2,20160901234500-2,20160901234500,1,ecigintelligenceinfo.com,http://ecigintelligenceinfo.com/2016/09/01/how...,NaN,NaN,ENV_SOLAR;WB_678_DIGITAL_GOVERNMENT;WB_694_BRO...,"GENERAL_GOVERNMENT,1765;TAX_ETHNICITY_INDIAN,1...",1#United States#US#US#38#-97#US;1#Madagascar#M...,...,"wc:301,c1.1:2,c12.1:16,c12.10:23,c12.12:7,c12....",NaN,NaN,NaN,https://youtube.com/watch?v=mIBTg7q9oNc;,1406|136||The annular eclipse is expected to o...,"South Africam Madagascar,129;South Atlantic Oc...",NaN,NaN,<PAGE_LINKS>http://live.slooh.com/stadium/live...
3,20160901234500-3,20160901234500,1,961theeagle.com,http://961theeagle.com/tags/geico/,NaN,NaN,WB_135_TRANSPORT;WB_1973_FINANCIAL_RISK_REDUCT...,"WB_1973_FINANCIAL_RISK_REDUCTION,83;WB_1973_FI...",NaN,...,"wc:165,c1.2:3,c12.1:16,c12.10:9,c12.12:4,c12.1...",http://961wodz.com/files/2015/01/wodzfmlogov2....,http://961theeagle.com/files/2013/02/pig-e1361...,NaN,https://youtube.com/subscribe_embed?bsv&usegap...,NaN,"New York State Department,32;Text Stop,111;New...","1000000,Moms,694;",NaN,<PAGE_LINKS>http://961wodz.com/tags/geico/;htt...
4,20160901234500-4,20160901234500,1,financialpost.com,http://business.financialpost.com/fp-comment/k...,NaN,NaN,LEADER;TAX_FNCACT;TAX_FNCACT_POLITICIANS;MANMA...,"ECON_FOREIGNINVEST,1166;ECON_FOREIGNINVEST,550...",1#United States#US#US#38#-97#US;1#India#IN#IN#...,...,"wc:944,c1.2:10,c1.3:2,c12.1:78,c12.10:138,c12....",http://wpmedia.business.financialpost.com/2014...,http://wpmedia.business.financialpost.com/2014...,NaN,NaN,5283|32||base erosion and profit shifting,"Kevin Libin,12;Elon Musk,169;Silicon Valley,32...","13000000000,euros,814;19000000000,dollars ,835...",NaN,<PAGE_LINKS>http://business.financialpost.com/...
5,20160901234500-5,20160901234500,1,nvi.com.au,http://www.nvi.com.au/story/4137790/mel-gibson...,NaN,NaN,TAX_ETHNICITY;TAX_ETHNICITY_AUSTRALIAN;TAX_FNC...,"TAX_ETHNICITY_ENGLISH,543;TAX_WORLDLANGUAGES_E...","3#Hollywood, California, United States#US#USCA...",...,"wc:400,c1.1:1,c1.2:1,c1.4:1,c12.1:31,c12.10:29...",http://nnimgt-a.akamaihd.net/transform/v1/crop...,NaN,NaN,NaN,2415|85||a common practice of the Holy Father ...,"Mel Gibson,41;Lethal Weapon,216;Jesus Christ I...",NaN,NaN,<PAGE_LINKS>http://www.hollywoodreporter.com/n...
6,20160901234500-6,20160901234500,1,wsbtradio.com,http://wsbtradio.com/sting-wants-people-to-be-...,NaN,NaN,SECURITY_SERVICES;TAX_FNCACT;TAX_FNCACT_POLICE...,"TAX_FNCACT_WOMAN,698;SECURITY_SERVICES,456;SEC...",NaN,...,"wc:225,c1.1:2,c1.4:1,c12.1:26,c12.10:19,c12.12...",http://i2.wp.com/wsbtradio.com/wp-content/uplo...,http://i2.wp.com/wsbtradio.com/wp-content/uplo...,NaN,NaN,NaN,"Wants People,21;His New,51;Stop Thinking About...",NaN,NaN,<PAGE_LINKS>http://www.sting.com/news/title/ho...
7,20160901234500-7,20160901234500,1,iheart.com,http://wnok.iheart.com/articles/trending-10465...,NaN,NaN,TAX_FNCACT;TAX_FNCACT_SINGER;USPEC_PO

In [27]:
datesToPull

['20160901234500',
 '20160902234500',
 '20160903234500',
 '20160904234500',
 '20160905234500',
 '20160906234500',
 '20160907234500',
 '20160908234500',
 '20160909234500',
 '20160910234500',
 '20160911234500',
 '20160912234500',
 '20160913234500',
 '20160914234500',
 '20160915234500',
 '20160916234500',
 '20160917234500',
 '20160918234500',
 '20160919234500',
 '20160920234500',
 '20160921234500',
 '20160922234500',
 '20160923234500',
 '20160924153000']

*****************

## Munging Data: Extracting Specific Datasets or all of them

Work with the returned GDELT data.  Specific whether we are pulling the `mentions`, `events`, or `gkg` date for the day or all.  

In [9]:
results = match_date(gdelt_timeString(dateInputCheck(date)))

In [10]:
target = results[2][results[2].str.contains('export')].reset_index(drop=True).ix[0]

In [11]:
target

'http://data.gdeltproject.org/gdeltv2/20160924150000.export.CSV.zip'

In [23]:
#############################################
# GDELT data download and extraction
#############################################

from StringIO import StringIO
import pandas as pd
import requests
import zipfile
import re

def downloadAndExtract(gdeltUrl):
    """Downloads and extracts GDELT zips without saving to disk"""
    
    response = requests.get(gdeltUrl, stream=True)
    zipdata = StringIO()
    zipdata.write(response.content)
    gdelt_zipfile = zipfile.ZipFile(zipdata,'r')
    name = re.search('(([\d]{4,}).*)',gdelt_zipfile.namelist()[0]).group().replace('.zip',"")
    data = gdelt_zipfile.read(name)
    gdelt_zipfile.close()
    del zipdata,gdelt_zipfile,name,response
    return pd.read_csv(StringIO(data),delimiter='\t',header=None)
    

def add_header(gdeltUrl):
    """Returns the header rows for the dataframe"""
    
    dbType = re.search(
        '(mentions|export|gkg)',
        gdeltUrl
        ).group()
    
    if dbType == "gkg":
        headers = gkgHeaders.tableId.tolist()
    
    elif dbType == "mentions":
        headers = mentionsHeaders.tableId.tolist()
        
    elif dbType == "export":
        headers = eventsDbHeaders.tableId.tolist()
        
    return headers

In [13]:
target

'http://data.gdeltproject.org/gdeltv2/20160924150000.export.CSV.zip'

In [14]:
gdelt_df = downloadAndExtract(target)
gdelt_df.columns = add_header(target)
gdelt_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1362 entries, 0 to 1361
Data columns (total 61 columns):
GLOBALEVENTID            1362 non-null int64
SQLDATE                  1362 non-null int64
MonthYear                1362 non-null int64
Year                     1362 non-null int64
FractionDate             1362 non-null float64
Actor1Code               1245 non-null object
Actor1Name               1245 non-null object
Actor1CountryCode        803 non-null object
Actor1KnownGroupCode     24 non-null object
Actor1EthnicCode         4 non-null object
Actor1Religion1Code      28 non-null object
Actor1Religion2Code      8 non-null object
Actor1Type1Code          621 non-null object
Actor1Type2Code          45 non-null object
Actor1Type3Code          1 non-null object
Actor2Code               1016 non-null object
Actor2Name               1016 non-null object
Actor2CountryCode        635 non-null object
Actor2KnownGroupCode     21 non-null object
Actor2EthnicCode         8 non-null object

In [15]:
gdelt_df

,GLOBALEVENTID,SQLDATE,MonthYear,Year,FractionDate,Actor1Code,Actor1Name,Actor1CountryCode,Actor1KnownGroupCode,Actor1EthnicCode,...,ActionGeo_Type,ActionGeo_FullName,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_ADM2Code,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,DATEADDED,SOURCEURL
0,582343584,20150925,201509,2015,2015.7260,NaN,NaN,NaN,NaN,NaN,...,4,"Jiquilillo, Chinandega, Nicaragua",NU,NU03,22430,12.731900,-87.44170,-1112100,20160924150000,http://azdailysun.com/news/local/community/fla...
1,582343585,20150925,201509,2015,2015.7260,AFR,AFRICA,AFR,NaN,NaN,...,4,"Osaka, Osaka, Japan",JA,JA32,35840,34.666700,135.50000,-240905,20160924150000,http://www.whio.com/news/world/burundi-thousan...
2,582343586,20150925,201509,2015,2015.7260,BRN,BRUNEIAN,BRN,NaN,NaN,...,1,Singapore,SN,SN,NaN,1.366700,103.80000,SN,20160924150000,http://health.asiaone.com/health/health-news/s...
3,582343587,20150925,201509,2015,2015.7260,CRM,TRAFFICKER,NaN,NaN,NaN,...,2,"Indiana, United States",US,USIN,NaN,39.864700,-86.26040,IN,20160924150000,https://www.indianagazette.com/news/reg-nation...
4,582343588,20150925,201509,2015,2015.7260,CVL,COMMUNITY,NaN,NaN,NaN,...,1,Canada,CA,CA,NaN,60.000000,-95.00000,CA,20160924150000,http://www.thecarillon.com/local/Community-can...
5,582343589,20150925,201509,2015,2015.7260,DOM,DOMINICAN REPUBLIC,DOM,NaN,NaN,...,4,"Loma Miranda, Dominican Republic (general), Do...",DR,DR00,36939,19.101100,-70.46460,-3362810,20160924150000,https://www.ncronline.org/preview/mining-our-m...
6,582343590,20150925,201509,2015,2015.7260,DOM,DOMINICAN REPUBLIC,DOM,NaN,NaN,...,4,"Vega Real, Duarte, Dominican Republic",DR,DR06,36897,19.250000,-70.25000,-3367035,20160924150000,https://www.ncronline.org/preview/mining-our-m...
7,582343591,20150925,201509,2015,2015.7260,FRA,FRANCE,FRA,NaN,NaN,...,4,"Paris, France (general), France",FR,FR00,16282,48.866700,2.33333,-1456928,20160924150000,http://ipolitics.ca/2016/09/24/can-trudeau-kee...
8,582343592,20150925,201509,2015,2015.7260,FRA,PARIS,FRA,NaN,NaN,...,4,"Ottawa, Ontario, Canada",CA,CA08,12755,45.416700,-75.70000,-570760,20160924150000,http://ipolitics.ca/2016/09/24/can-trudeau-kee...
9,582343593,20150925,201509,2015,2015.7260,GOV,INTERIOR MINIST,NaN,NaN,NaN,...,4,"Paris, France (general), France",FR,FR00,16282,48.866700,2.33333,-1456928,20160924150000,http://www.mfs-theothernews.com/search?updated...


In [237]:
combined = gdelt_df.merge(gdelt_df2,how='outer',on='GLOBALEVENTID')

In [238]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11767 entries, 0 to 11766
Data columns (total 76 columns):
GLOBALEVENTID                11767 non-null float64
SQLDATE                      2785 non-null float64
MonthYear                    2785 non-null float64
Year                         2785 non-null float64
FractionDate                 2785 non-null float64
Actor1Code                   2550 non-null object
Actor1Name                   2550 non-null object
Actor1CountryCode            1573 non-null object
Actor1KnownGroupCode         63 non-null object
Actor1EthnicCode             31 non-null object
Actor1Religion1Code          55 non-null object
Actor1Religion2Code          13 non-null object
Actor1Type1Code              1220 non-null object
Actor1Type2Code              68 non-null object
Actor1Type3Code              0 non-null float64
Actor2Code                   2005 non-null object
Actor2Name                   2005 non-null object
Actor2CountryCode            1246 non-null obje

In [240]:
combined.columns

Index([u'GLOBALEVENTID', u'SQLDATE', u'MonthYear', u'Year', u'FractionDate',
       u'Actor1Code', u'Actor1Name', u'Actor1CountryCode',
       u'Actor1KnownGroupCode', u'Actor1EthnicCode', u'Actor1Religion1Code',
       u'Actor1Religion2Code', u'Actor1Type1Code', u'Actor1Type2Code',
       u'Actor1Type3Code', u'Actor2Code', u'Actor2Name', u'Actor2CountryCode',
       u'Actor2KnownGroupCode', u'Actor2EthnicCode', u'Actor2Religion1Code',
       u'Actor2Religion2Code', u'Actor2Type1Code', u'Actor2Type2Code',
       u'Actor2Type3Code', u'IsRootEvent', u'EventCode', u'EventBaseCode',
       u'EventRootCode', u'QuadClass', u'GoldsteinScale', u'NumMentions',
       u'NumSources', u'NumArticles', u'AvgTone', u'Actor1Geo_Type',
       u'Actor1Geo_FullName', u'Actor1Geo_CountryCode', u'Actor1Geo_ADM1Code',
       u'Actor1Geo_ADM2Code', u'Actor1Geo_Lat', u'Actor1Geo_Long',
       u'Actor1Geo_FeatureID', u'Actor2Geo_Type', u'Actor2Geo_FullName',
       u'Actor2Geo_CountryCode', u'Actor2Geo_ADM1Cod

In [246]:
# combined.[(combined.Confidence != None) & (combined.MonthYear != None)]
combined[['Actor1Code','Actor1Name']][(combined.GoldsteinScale <= -5.2) & (combined.Actor1Code != "")].fillna('')

,Actor1Code,Actor1Name
36,LBNGOV,TYRE
37,LBNGOV,TYRE
46,USA,UNITED STATES
55,LBN,TYRE
56,LBN,TYRE
67,USA,UNITED STATES
81,,
111,,
137,,
138,,


# Early Pipeline to Write out R Dataframe


Ways to install
```python
pip install feather-format
```

```bash
conda install feather-format -c conda-forge
```


###  **IT WORKS!!!**

In [47]:
import feather
path = 'my_data.feather'
feather.api.write_dataframe(testdf, path)
newtestdf = feather.api.read_dataframe(path)

# Leftovers; Junkyard below (stuff to work on)

In [ ]:
results = masterListdf[masterListdf[2].str.contains(gdelt_timeString(dateInputCheck(date)))==True]

In [ ]:
results[2].reset_index().ix[0][2]

In [ ]:
results[results[2].str.contains('gkg')]

In [ ]:
gdelt_timeString(dateInputCheck(date))

In [ ]:
import re
from dateutil.parser import parse
re.search('(([\d]{4,}).*)',clean[20][-1]).group()

In [ ]:
if bool(4>3):
    print "Hello"

In [ ]:
(datetime.datetime.now().replace(hour=0,minute=0,second=0,microsecond=0)) == parse("2016 09 18" )

In [ ]:
b = dateutil.parser.parse(re.search('([\d]{4,})',clean[20][-1]).group())

In [ ]:
matchDate = re.search('([\d]{4,})',clean[20][-1]).group()

In [ ]:
def time_change(current,diff):
    date = current.replace(minute=0, second=0) + timedelta(minutes=diff)
    return date.strftime("%Y%m%d%H%M%S")
    

In [ ]:
# pulling most current daily report

import numpy as np
import datetime
from datetime import timedelta

currentTime = datetime.datetime.now()
timeDiff = currentTime.minute / 15 

query = np.where(timeDiff == 1,time_change(currentTime,diff=15),
        np.where(timeDiff == 2, time_change(currentTime,diff=30),
                 np.where(timeDiff == 3, time_change(currentTime,diff=45),
                          time_change(currentTime,diff=0))))

baseUrl = 'http://data.gdeltproject.org/gdeltv2/' + str(query) + '.export.CSV.zip'

In [ ]:
data

In [ ]:
myzipfile.namelist()

In [ ]:

import zipfile


r = requests.get(baseUrl, stream=True)

# with open('gdelt.zip', 'wb') as f:
#     f.write(r.content)
# fh = open('gdelt.zip')
# g = zipfile.ZipFile(fh)
# g.extractall()

from StringIO import StringIO
zipdata = StringIO()
zipdata.write(r.content)
myzipfile = zipfile.ZipFile(zipdata,'r')
data = myzipfile.read(str(query) + '.export.CSV')
gdeltdf = pd.read_csv(StringIO(data),delimiter='\t',header=None)


In [ ]:
gdeltdf.columns=headers.tableId.tolist()

In [ ]:
gdeltdf.SOURCEURL[((gdeltdf.ActionGeo_CountryCode =='SY')|(gdeltdf.ActionGeo_CountryCode =='IZ')) & (gdeltdf.GoldsteinScale < -4)]

In [49]:
text = '''
GLOBALEVENTID	INTEGER	NULLABLE	This is the ID of the event that was mentioned in the article.
EventTimeDate	INTEGER	NULLABLE	This is the 15-minute timestamp (YYYYMMDDHHMMSS) when the event being mentioned was first recorded by GDELT (the DATEADDED field of the original event record).  This field can be compared against the next one to identify events being mentioned for the first time (their first mentions) or to identify events of a particular vintage being mentioned now (such as filtering for mentions of events at least one week old).
MentionTimeDate	INTEGER	NULLABLE	This is the 15-minute timestamp (YYYYMMDDHHMMSS) of the current update.  This is identical for all entries in the update file but is included to make it easier to load the Mentions table into a database.
MentionType	INTEGER	NULLABLE	This is a numeric identifier that refers to the source collection the document came from and is used to interpret the MentionIdentifier in the next column.  In essence, it specifies how to interpret the MentionIdentifier to locate the actual document.  At present, it can hold one of the following values:o 1 = WEB (The document originates from the open web and the MentionIdentifier is a fully-qualified URL that can be used to access the document on the web).o 2 = CITATIONONLY (The document originates from a broadcast, print, or other offline source in which only a textual citation is available for the document.  In this case the MentionIdentifier contains the textual citation for the document).o 3 = CORE (The document originates from the CORE archive and the MentionIdentifier contains its DOI, suitable for accessing the original document through the CORE website).o 4 = DTIC (The document originates from the DTIC archive and the MentionIdentifier contains its DOI, suitable for accessing the original document through the DTIC website).o 5 = JSTOR (The document originates from the JSTOR archive and the MentionIdentifier contains its DOI, suitable for accessing the original document through your JSTOR subscription if your institution subscribes to it).o 6 = NONTEXTUALSOURCE (The document originates from a textual proxy (such as closed captioning) of a non-textual information source (such as a video) available via a URL and the MentionIdentifier provides the URL of the non-textual original source.  At present, this Collection Identifier is used for processing of the closed captioning streams of the Internet Archive Television News Archive in which each broadcast is available via a URL, but the URL offers access only to the video of the broadcast and does not provide any access to the textual closed captioning used to generate the metadata.  This code is used in order to draw a distinction between URL-based textual material (Collection Identifier 1 (WEB) and URL-based non-textual material like the Television News Archive).
MentionSourceName	STRING	NULLABLE	This is a human-friendly identifier of the source of the document.  For material originating from the open web with a URL this field will contain the top-level domain the page was from.  For BBC Monitoring material it will contain “BBC Monitoring” and for JSTOR material it will contain “JSTOR.”  This field is intended for human display of major sources as well as for network analysis of information flows by source, obviating the requirement to perform domain or other parsing of the MentionIdentifier field.
MentionIdentifier	STRING	NULLABLE	This is the unique external identifier for the source document.  It can be used to uniquely identify the document and access it if you have the necessary subscriptions or authorizations and/or the document is public access.  This field can contain a range of values, from URLs of open web resources to textual citations of print or broadcast material to DOI identifiers for various document repositories.  For example, if MentionType is equal to 1, this field will contain a fully-qualified URL suitable for direct access.  If MentionType is equal to 2, this field will contain a textual citation akin to what would appear in an academic journal article referencing that document (NOTE that the actual citation format will vary (usually between APA, Chicago, Harvard, or MLA) depending on a number of factors and no assumptions should be made on its precise format at this time due to the way in which this data is currently provided to GDELT – future efforts will focus on normalization of this field to a standard citation format).  If MentionType is 3, the field will contain a numeric or alpha-numeric DOI that can be typed into JSTOR’s search engine to access the document if your institution has a JSTOR subscription.
SentenceID	INTEGER	NULLABLE	The sentence within the article where the event was mentioned (starting with the first sentence as 1, the second sentence as 2, the third sentence as 3, and so on).  This can be used similarly to the CharOffset fields below, but reports the event’s location in the article in terms of sentences instead of characters, which is more amenable to certain measures of the “importance” of an event’s positioning within an article.
Actor1CharOffset	INTEGER	NULLABLE	The location within the article (in terms of English characters) where Actor1 was found.  This can be used in combination with the GKG or other analysis to identify further characteristics and attributes of the actor.  NOTE: due to processing performed on each article, this may be slightly offset from the position seen when the article is rendered in a web browser.
Actor2CharOffset	INTEGER	NULLABLE	The location within the article (in terms of English characters) where Actor2 was found.  This can be used in combination with the GKG or other analysis to identify further characteristics and attributes of the actor.  NOTE: due to processing performed on each article, this may be slightly offset from the position seen when the article is rendered in a web browser.
ActionCharOffset	INTEGER	NULLABLE	The location within the article (in terms of English characters) where the core Action description was found.  This can be used in combination with the GKG or other analysis to identify further characteristics and attributes of the actor.  NOTE: due to processing performed on each article, this may be slightly offset from the position seen when the article is rendered in a web browser.
InRawText	INTEGER	NULLABLE	This records whether the event was found in the original unaltered raw article text (a value of 1) or whether advanced natural language processing algorithms were required to synthesize and rewrite the article text to identify the event (a value of 0).  See the discussion on the Confidence field below for more details.  Mentions with a value of “1” in this field likely represent strong detail-rich references to an event.
Confidence	INTEGER	NULLABLE	Percent confidence in the extraction of this event from this article.  See the discussion in the codebook at http://data.gdeltproject.org/documentation/GDELT-Event_Codebook-V2.0.pdf
MentionDocLen	INTEGER	NULLABLE	The length in English characters of the source document (making it possible to filter for short articles focusing on a particular event versus long summary articles that casually mention an event in passing).
MentionDocTone	FLOAT	NULLABLE	The same contents as the AvgTone field in the Events table, but computed for this particular article.  NOTE: users interested in emotional measures should use the MentionIdentifier field above to merge the Mentions table with the GKG table to access the complete set of 2,300 emotions and themes from the GCAM system.
MentionDocTranslationInfo	STRING	NULLABLE	This field is internally delimited by semicolons and is used to record provenance information for machine translated documents indicating the original source language and the citation of the translation system used to translate the document for processing.  It will be blank for documents originally in English.  At this time the field will also be blank for documents translated by a human translator and provided to GDELT in English (such as BBC Monitoring materials) – in future this field may be expanded to include information on human translation pipelines, but at present it only captures information on machine translated materials.  An example of the contents of this field might be “srclc:fra; eng:Moses 2.1.1 / MosesCore Europarl fr-en / GT-FRA 1.0”.  NOTE:  Machine translation is often not as accurate as human translation and users requiring the highest possible confidence levels may wish to exclude events whose only mentions are in translated reports, while those needing the highest-possible coverage of the non-Western world will find that these events often offer the earliest glimmers of breaking events or smaller-bore events of less interest to Western media.o SRCLC. This is the Source Language Code, representing the three-letter ISO639-2 code of the language of the original source material. o ENG.  This is a textual citation string that indicates the engine(s) and model(s) used to translate the text.  The format of this field will vary across engines and over time and no expectations should be made on the ordering or formatting of this field.  In the example above, the string “Moses 2.1.1 / MosesCore Europarl fr-en / GT-FRA 1.0” indicates that the document was translated using version 2.1.1 of the Moses   SMT platform, using the “MosesCore Europarl fr-en” translation and language models, with the final translation enhanced via GDELT Translingual’s own version 1.0 French translation and language models.  A value of “GT-ARA 1.0” indicates that GDELT Translingual’s version 1.0 Arabic translation and language models were the sole resources used for translation.  Additional language systems used in the translation pipeline such as word segmentation systems are also captured in this field such that a value of “GT-ZHO 1.0 / Stanford PKU” indicates that the Stanford Chinese Word Segmenter   was used to segment the text into individual words and sentences, which were then translated by GDELT Translingual’s own version 1.0 Chinese (Traditional or Simplified) translation and language models.
Extras	STRING	NULLABLE	This field is currently blank, but is reserved for future use to encode special additional measurements for selected material.
'''

In [50]:
from StringIO import StringIO
eventMentions = pd.read_csv(StringIO(text),delimiter='\t',header=None)

In [51]:
eventMentions.columns=['tableId', 'dataType','Empty', 'Description']

In [53]:
eventMentions.to_csv('../../gdelt2HeaderRows/schema_csvs/GDELT_2.0_eventMentions_Column_Labels_Header_Row_Sep2016.tsv',encoding='utf-8',sep='\t')

In [48]:
eventMentions

,tableId,dataType,Empty,Description
0,GLOBALEVENTID,INTEGER,NULLABLE,This is the ID of the event that was mentioned...
1,EventTimeDate,INTEGER,NULLABLE,This is the 15-minute timestamp (YYYYMMDDHHMMS...
2,MentionTimeDate,INTEGER,NULLABLE,This is the 15-minute timestamp (YYYYMMDDHHMMS...
3,MentionType,INTEGER,NULLABLE,This is a numeric identifier that refers to th...
4,MentionSourceName,STRING,NULLABLE,This is a human-friendly identifier of the sou...
5,MentionIdentifier,STRING,NULLABLE,This is the unique external identifier for the...
6,SentenceID,INTEGER,NULLABLE,The sentence within the article where the even...
7,Actor1CharOffset,INTEGER,NULLABLE,The location within the article (in terms of E...
8,Actor2CharOffset,INTEGER,NULLABLE,The location within the article (in terms of E...
9,ActionCharOffset,INTEGER,NULLABLE,The location within the article (in terms of E...


In [ ]:
gkgdf.to_csv('../../gdelt2HeaderRows/schema_csvs/GDELT_2.0_gdeltKnowledgeGraph_Column_Labels_Header_Row_Sep2016.tsv',encoding='utf-8',sep='\t')

In [ ]:
gkgdf.to_csv('GDELT_2.0_gdeltKnowledgeGraph_Column_Labels_Header_Row_Sep2016.csv',sep='\t',index=False,encoding='utf-8')

In [ ]:
headers.to_csv('GDELT_2.0_Events_Column_Labels_Header_Row_Sep2016.csv', index=False,encoding='utf-8')

In [ ]:
import pandas as pd
mentionsdf = pd.read_csv(StringIO(text),delimiter='\t',header=None)
mentionsdf.columns=headers.columns.tolist()